# Advanced Certification Programme in AI and MLOps
## A programme by IISc and TalentSprint
### Mini-Project Notebook: Structured Data Classification

## Problem Statement

To predict whether a patient has a heart disease.

## Learning Objectives

At the end of the experiment, you will be able to

* understand the Cleveland Clinic Foundation for Heart Disease dataset
* pre-process this dataset
* build a neural network architecture/model using Keras sequential or functional api
* perform model training
* perform inference on an unseen data
* build a Gradio interface for this application

## Introduction

This example demonstrates how to do structured data classification, starting from a raw
CSV file. Our data includes both numerical and categorical features. We will do preprocessing to normalize the numerical features and vectorize the categorical
ones.

### Dataset

[Our dataset](https://archive.ics.uci.edu/ml/datasets/heart+Disease) is provided by the
Cleveland Clinic Foundation for Heart Disease.
It's a CSV file with 303 rows. Each row contains information about a patient (a
**sample**), and each column describes an attribute of the patient (a **feature**). We
use the features to predict whether a patient has a heart disease (**binary
classification**).

Here's the description of each feature:

Column| Description| Feature Type
------------|--------------------|----------------------
Age | Age in years | Numerical
Sex | (1 = male; 0 = female) | Categorical
CP | Chest pain type (0, 1, 2, 3, 4) | Categorical
Trestbpd | Resting blood pressure (in mm Hg on admission) | Numerical
Chol | Serum cholesterol in mg/dl | Numerical
FBS | fasting blood sugar in 120 mg/dl (1 = true; 0 = false) | Categorical
RestECG | Resting electrocardiogram results (0, 1, 2) | Categorical
Thalach | Maximum heart rate achieved | Numerical
Exang | Exercise induced angina (1 = yes; 0 = no) | Categorical
Oldpeak | ST depression induced by exercise relative to rest | Numerical
Slope | Slope of the peak exercise ST segment | Numerical
CA | Number of major vessels (0-3) colored by fluoroscopy | Both numerical & categorical
Thal | 3 = normal; 6 = fixed defect; 7 = reversible defect | Categorical
Target | Diagnosis of heart disease (1 = true; 0 = false) | Target

In [ ]:
#@title Download the data
!wget -qq https://cdn.iisc.talentsprint.com/AIandMLOps/Datasets/heart.csv
print("Data Downloaded Successfuly!!")
!ls | grep '.csv'

Data Downloaded Successfuly!!
heart.csv
heart.csv.1
heart.csv.2
heart.csv.3
heart.csv.4
heart.csv.5


## Grading = 10 Points

### Import Required Packages

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from sklearn.preprocessing import OrdinalEncoder as OrdinalEncoder_Sk
from sklearn.pipeline import Pipeline
from sklearn.impute  import SimpleImputer
from sklearn.feature_selection import SelectFromModel

## Load the data and pre-process it [3 Marks]

### Load data into a Pandas dataframe

Hint:: pd.read_csv

In [ ]:
file_url = "/content/heart.csv"
## YOUR CODE HERE
data = pd.read_csv(file_url)

Check the shape of the dataset:

In [ ]:
## YOUR CODE HERE
data.shape

(303, 14)

Check the preview of a few samples:

Hint:: head()

In [ ]:
## YOUR CODE HERE
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


Draw some inference from the data. What does the target column indicate?

The last column, "target", indicates whether the patient has a heart disease (1) or not
(0).

### Missing values

In [ ]:
# Check if any missing values is present
## YOUR CODE HERE
#data.isnull().sum().values
summary = pd.DataFrame(data.dtypes,columns=['dtype'])
summary=summary.reset_index();
summary=summary.rename(columns={'index':'Name'})
summary['Null_counts']=data.isnull().sum().values
summary['Uniques']=data.nunique().values
summary['Null_percent']=summary['Null_counts']*100/len(data)
summary.sort_values(by='Null_percent',ascending=False,inplace=True)
summary

,Name,dtype,Null_counts,Uniques,Null_percent
0,age,int64,0,41,0.0
1,sex,int64,0,2,0.0
2,cp,int64,0,5,0.0
3,trestbps,int64,0,50,0.0
4,chol,int64,0,152,0.0
5,fbs,int64,0,2,0.0
6,restecg,int64,0,3,0.0
7,thalach,int64,0,91,0.0
8,exang,int64,0,2,0.0
9,oldpeak,float64,0,40,0.0


### Show the unique values present in each categorical columns

- Remove the rows which has '1' and '2' as values in `thal` column

In [ ]:
# Show all the columns in dataframe
## YOUR CODE HERE
print(data.columns.tolist())

['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']


In [ ]:
# Print the unique values present in each categorical columns

categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'ca', 'thal']

## YOUR CODE HERE
#for var in categorical_cols.columns
#data[categorical_cols].nunique().values
for var in categorical_cols:
  unique_data = data[var].unique()
  unique_count = data[var].nunique()
  print(f"unique value in column'{var}':'{unique_data}':'{unique_count}'")

unique value in column'sex':'[1 0]':'2'
unique value in column'cp':'[1 4 3 2 0]':'5'
unique value in column'fbs':'[1 0]':'2'
unique value in column'restecg':'[2 0 1]':'3'
unique value in column'exang':'[0 1]':'2'
unique value in column'ca':'[0 3 2 1]':'4'
unique value in column'thal':'['fixed' 'normal' 'reversible' '1' '2']':'5'


In [ ]:
# Print the unique values present in each categorical columns along with their counts

## YOUR CODE HERE


- Remove the rows which has '1' and '2' as values in `thal` column

In [ ]:
# Find indices of the rows which has '1', '2' as values in `thal` column
dataCopy = data.copy()
print(data['thal'].isin(['1','2']))
#    idx = ## YOUR CODE HERE
idx = data[data['thal'].isin(['1','2'])].index
#print(data['thal'][253])
print(idx)

0      False
1      False
2      False
3      False
4      False
       ...  
298    False
299    False
300    False
301    False
302    False
Name: thal, Length: 303, dtype: bool
Index([247, 252], dtype='int64')


In [ ]:
# Drop the above indexed rows

## YOUR CODE HERE
print(data.shape)
updated_data = data.drop(idx)
print(updated_data.shape)
updated_data.head()

(303, 14)
(301, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [ ]:
# Recheck the unique values present in each categorical columns
#updated_data.head()
## YOUR CODE HERE
for var in categorical_cols:
  updated_unique_data = updated_data[var].unique()
  print(updated_unique_data)



[1 0]
[1 4 3 2 0]
[1 0]
[2 0 1]
[0 1]
[0 3 2 1]
['fixed' 'normal' 'reversible']


### Convert the categorical values present in `thal` column to numerical labels

Hint: Create a dictionary mapping

In [ ]:
cat_to_num_dict={'fixed':0,'normal':1,'reversible':2}
updated_data['thal']=updated_data['thal'].replace(cat_to_num_dict)
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'ca', 'thal']
for cols in categorical_cols:
  print("unique values in categorical col:",cols)
  print(updated_data[cols].unique())


unique values in categorical col: sex
[1 0]
unique values in categorical col: cp
[1 4 3 2 0]
unique values in categorical col: fbs
[1 0]
unique values in categorical col: restecg
[2 0 1]
unique values in categorical col: exang
[0 1]
unique values in categorical col: ca
[0 3 2 1]
unique values in categorical col: thal
[0 1 2]


<ipython-input-212-2277439f9889>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  updated_data['thal']=updated_data['thal'].replace(cat_to_num_dict)


Alternate Approach

In [ ]:
## YOUR CODE HERE
#numerical_data = pd.get_dummies(updated_data['thal'])
#numerical_data.head()
from sklearn import preprocessing

#make an instance of Label Encoder
label_encoder = preprocessing.LabelEncoder()
updated_data['thal'] = label_encoder.fit_transform(updated_data['thal'])
updated_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,1,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,2,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,1,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,1,0


Alternate Approach

In [ ]:
'''
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute  import SimpleImputer
numerical_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope']
# Define transformers for numerical and categorical columns
numerical_transformer = Pipeline(steps=[
    #('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    #('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal_enc',OrdinalEncoder_Sk())
])

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])
## YOUR CODE HERE

# Create the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

X_processed = pipeline.fit_transform(updated_data)

# Extracting feature names for numerical columns
num_feature_names = numerical_cols

# Extracting feature names for categorical columns after one-hot encoding
cat_encoder = pipeline.named_steps['preprocessor'].named_transformers_['cat'].named_steps['ordinal_enc']
cat_feature_names = cat_encoder.get_feature_names_out(categorical_cols)

# Concatenating numerical and categorical feature names
feature_names = num_feature_names + list(cat_feature_names)

X_processed_df = pd.DataFrame(X_processed,columns=feature_names)
y = updated_data.target
X_processed_df.head()

'''

"\nfrom sklearn.compose import ColumnTransformer\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.impute  import SimpleImputer\nnumerical_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope']\n# Define transformers for numerical and categorical columns\nnumerical_transformer = Pipeline(steps=[\n    #('imputer', SimpleImputer(strategy='mean')),\n    ('scaler', StandardScaler())\n])\n\ncategorical_transformer = Pipeline(steps=[\n    #('imputer', SimpleImputer(strategy='most_frequent')),\n    ('ordinal_enc',OrdinalEncoder_Sk())\n])\n\n# Combine transformers using ColumnTransformer\npreprocessor = ColumnTransformer(\n    transformers=[\n        ('num', numerical_transformer, numerical_cols),\n        ('cat', categorical_transformer, categorical_cols)\n    ])\n## YOUR CODE HERE\n\n# Create the pipeline\npipeline = Pipeline(steps=[('preprocessor', preprocessor)])\n\nX_processed = pipeline.fit_transform(updated_data)\n\n# Extracting feature names for numerical colum

### Split the dataset into training and testing sets

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(updated_data.drop('target', axis=1), updated_data.target, test_size=0.1, random_state=42, stratify=y)

In [ ]:
from sklearn.model_selection import train_test_split

## YOUR CODE HERE (perform stratified sampling/splitting)
#X_train, X_test, y_train, y_test = train_test_split(updated_data.drop('target', axis=1), # predictors
#                                                   updated_data.target, # target
#                                                    test_size=0.1,
#                                                    random_state=0)  # for reproducibility

### Scale the numerical features

In [ ]:
numerical_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope']

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)
#df = pd.DataFrame(X_train,columns=['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'])
#df.head()
#df['age'].describe()

## Building the model [3 Marks]

* Use tf.keras.layers.Input() for input layer
* Add dense layers
* Add dropout layers
* Add a classification layer at the end


In [ ]:
#print(X_train.to_numpy().reshape(270*13,1).shape)
print(X_train.shape[0])

270


In [ ]:
# Create model
from keras.layers import Input
inputs = Input(shape=(X_train.shape[1],))
## YOUR CODE HERE
dense_layer = tf.keras.layers.Dense(units=32, activation='relu')(inputs)
dense_layer = tf.keras.layers.Dense(units=32, activation='relu')(dense_layer)
dense_layer = tf.keras.layers.Dropout(0.5)(dense_layer)
output_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(dense_layer)
model = tf.keras.Model(inputs=inputs, outputs=output_layer)
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)          │ (None, 13)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 32)                  │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,537 (6.00 KB)

 Trainable params: 1,537 (6.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile model with 'adam' optimizer, appropriate loss and metric
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
## YOUR CODE HERE

NameError: name 'model' is not defined

In [ ]:
# Perform training
epochs=50
batch_size=32
validation_split=0.2

model.fit(x= X_train, y=y_train,epochs=50,batch_size=32,validation_split=0.2)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/losses/losses.py:27: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - accuracy: 0.6085 - loss: 0.0000e+00 - val_accuracy: 0.6852 - val_loss: 0.0000e+00
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6884 - loss: 0.0000e+00 - val_accuracy: 0.6852 - val_loss: 0.0000e+00
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7266 - loss: 0.0000e+00 - val_accuracy: 0.6852 - val_loss: 0.0000e+00
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7254 - loss: 0.0000e+00 - val_accuracy: 0.6852 - val_loss: 0.0000e+00
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7454 - loss: 0.0000e+00 - val_accuracy: 0.6852 - val_loss: 0.0000e+00
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7248 - loss: 0.0000e+00 - val_accuracy: 0.6852 - val_loss: 0.0000e+00
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7689 - loss: 0.0000e+00 - val_accuracy: 0.6852 - val_loss: 0.0000e+00
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7880 - loss: 0.0000e+00

In [ ]:
# Performance on test set
#X_test_processed = pipeline.fit_transform(X_test)
scores =model.evaluate(X_test,y_test)
scores[1]
model.save('nn_model.keras')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.7097 - loss: 0.0000e+00


## Inference on new data [1 Mark]

To get a prediction for a new sample, you can simply call `model.predict()`.

In [ ]:
# Inference on new data

sample = {
    "age": 60,
    "sex": 1,
    "cp": 1,
    "trestbps": 145,
    "chol": 233,
    "fbs": 1,
    "restecg": 2,
    "thalach": 150,
    "exang": 0,
    "oldpeak": 2.3,
    "slope": 3,
    "ca": 0,
    "thal": "fixed",
}


In [ ]:
import json
from sklearn import preprocessing
from tensorflow.keras.models import load_model
json_dump = json.dumps(sample)
json_data = json.loads(json_dump)
sample_df = pd.DataFrame([json_data])
#sample_df.head()


cat_to_num_dict={'fixed':0,'normal':1,'reversible':2}
sample_df['thal']=sample_df['thal'].replace(cat_to_num_dict)

#make an instance of Label Encoder
#label_encoder = preprocessing.LabelEncoder()
#sample_df['thal'] = label_encoder.fit_transform(sample_df['thal'])
sample_df.head()
print(sample_df.dtypes)
savedModel=tf.keras.models.load_model("/content/nn_model.keras")
y_pred = savedModel.predict(sample_df)
print(f"'output is ':{y_pred}")

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
dtype: object
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


<ipython-input-225-64eaa3516e9a>:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sample_df['thal']=sample_df['thal'].replace(cat_to_num_dict)


'output is ':[[0.]]


In [ ]:
'''
import json
from tensorflow.keras.models import load_model
json_dump = json.dumps(sample)
json_data = json.loads(json_dump)
sample_df = pd.DataFrame([json_data])
sample_df.head()
X_json__processed = pipeline.fit_transform(sample_df)

# Extracting feature names for numerical columns
num_feature_names = numerical_cols

# Extracting feature names for categorical columns after one-hot encoding
cat_encoder = pipeline.named_steps['preprocessor'].named_transformers_['cat'].named_steps['ordinal_enc']
cat_feature_names = cat_encoder.get_feature_names_out(categorical_cols)

# Concatenating numerical and categorical feature names
feature_names = num_feature_names + list(cat_feature_names)

X_json_processed_df = pd.DataFrame(X_json__processed,columns=feature_names)
y = updated_data.target
X_json_processed_df.head()
savedModel=tf.keras.models.load_model("/content/nn_model.keras")
#savedModel=model.load_weights("/content/NNModel.weights.h5")
#savedModel.summary()
#y_pred=savedModel.predict(X_json_processed_df)
y_pred=savedModel.predict(sample_df)

print(y_pred)
'''

'\nimport json\nfrom tensorflow.keras.models import load_model\njson_dump = json.dumps(sample)\njson_data = json.loads(json_dump)\nsample_df = pd.DataFrame([json_data])\nsample_df.head()\nX_json__processed = pipeline.fit_transform(sample_df)\n\n# Extracting feature names for numerical columns\nnum_feature_names = numerical_cols\n\n# Extracting feature names for categorical columns after one-hot encoding\ncat_encoder = pipeline.named_steps[\'preprocessor\'].named_transformers_[\'cat\'].named_steps[\'ordinal_enc\']\ncat_feature_names = cat_encoder.get_feature_names_out(categorical_cols)\n\n# Concatenating numerical and categorical feature names\nfeature_names = num_feature_names + list(cat_feature_names)\n\nX_json_processed_df = pd.DataFrame(X_json__processed,columns=feature_names)\ny = updated_data.target\nX_json_processed_df.head()\nsavedModel=tf.keras.models.load_model("/content/nn_model.keras")\n#savedModel=model.load_weights("/content/NNModel.weights.h5")\n#savedModel.summary()\n#y_

## Gradio Implementation [3 Marks]

Create a Gradio interface for this `Heart Disease Prediction` application. For the feature values given by the user as input, perform predcition using the trained model, and return the result back to user.

Make use of gradio elements such as Textbox, Radio buttons, etc.

In [ ]:
%%capture
!pip -q install gradio

In [ ]:
import gradio
import gradio as gr

In [ ]:
# UI - Input components
## YOUR CODE HERE ...
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            age = gr.Number(label='Age',value=0)
            sex = gr.Dropdown(label='sex',choices=[0,1],value=[])
            cp = gr.Number(label='cp')
            trestbps = gr.Number(label='trestbps',value=0)
            chol = gr.Number(label='chol',value=0)
            fbs = gr.Number(label='fbs',value=0)
            restecg = gr.Number(label='restecg',value=0)
            thalach = gr.Number(label='thalach',value=0)
            exang = gr.Number(label='exang',value=0)
            oldpeak = gr.Number(label='oldpeak',value=0)
            slope = gr.Number(label='slope',value=0)
            ca = gr.Number(label='ca',value=0)
            thal = gr.Dropdown(label='thal',choices=['normal','fixed','reversible'],value=[])
            submit_btn = gr.Button(value="Predict")
            #with gr.Column():
            result = gr.Label()

#if __name__ == "__main__":
 #   demo.launch(share=True,show_api=False)

# UI - Output component
## YOUR CODE HERE ...


In [ ]:
# Label prediction function

## YOUR CODE HERE
def predict(age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal):

  input_df=pd.DataFrame([{
      'age':age,
      'sex':sex,
      'cp':cp,
      'trestbps':trestbps,
      'chol':chol,
      'fbs':fbs,
      'restecg':restecg,
      'thalach':thalach,
      'exang':exang,
      'oldpeak':oldpeak,
      'slope':slope,
      'ca':ca,
      'thal':thal
  }])

  cat_to_num_dict={'fixed':0,'normal':1,'reversible':2}
  input_df['thal']=input_df['thal'].replace(cat_to_num_dict)
  print(input_df.dtypes)
  # Load the model
  #model = tf.keras.models.load_model("/content/nn_model.keras")
  #model.summary()
  #df_np = input_df.to_numpy()
  #tf_data = tf.convert_to_tensor(df_np)
  y_predict=model.predict(x=input_df)
  return y_predict[0]
#predict(60, 1,1,145,233,1,2,150,0, 2.3,3, 0,"fixed")

In [ ]:
# Create Gradio interface object and launch it with (share=True)
# Create the Gradio interface
iface = gr.Interface(
   fn=predict,
   inputs=[age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal],
   outputs='text',
  title="Heart attack prediction"
)
## YOUR CODE HERE
iface.launch(debug=True,share=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://03a07c1cc59b1abbbf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


age         int64
sex         int64
cp          int64
trestbps    int64
chol        int64
fbs         int64
restecg     int64
thalach     int64
exang       int64
oldpeak     int64
slope       int64
ca          int64
thal        int64
dtype: object
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step


<ipython-input-234-a6f325755436>:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_df['thal']=input_df['thal'].replace(cat_to_num_dict)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7870 <> https://03a07c1cc59b1abbbf.gradio.live


In [ ]:
import gradio as gr
import pandas as pd
import tensorflow as tf

# Load the model
model = tf.keras.models.load_model("/content/nn_model.keras")

def predict(age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpeak, slope, ca, thal):
    # Prepare the input data in a DataFrame
    input_df = pd.DataFrame([{
        'age': age,
        'sex': sex,
        'cp': cp,
        'trestbps': trestbps,
        'chol': chol,
        'fbs': fbs,
        'restecg': restecg,
        'thalach': thalach,
        'exang': exang,
        'oldpeak': oldpeak,
        'slope': slope,
        'ca': ca,
        'thal': thal
    }])

    # Map categorical 'thal' column values to numerical values
    cat_to_num_dict = {'fixed': 0, 'normal': 1, 'reversible': 2}
    input_df['thal'] = input_df['thal'].replace(cat_to_num_dict)

    # Make prediction
    y_predict = model.predict(input_df)
    return f"Predicted Risk: {y_predict[0][0]:.2f}"

# Define the Gradio interface
iface = gr.Interface(
    fn=predict,
    inputs=[
        gr.inputs.Number(label="Age"),
        gr.inputs.Radio(choices=["male", "female"], label="Sex"),
        gr.inputs.Radio(choices=[0, 1, 2, 3], label="Chest Pain Type (cp)"),
        gr.inputs.Number(label="Resting Blood Pressure (trestbps)"),
        gr.inputs.Number(label="Serum Cholesterol (chol)"),
        gr.inputs.Radio(choices=[0, 1], label="Fasting Blood Sugar (fbs)"),
        gr.inputs.Radio(choices=[0, 1, 2], label="Resting Electrocardiographic Results (restecg)"),
        gr.inputs.Number(label="Max Heart Rate (thalach)"),
        gr.inputs.Radio(choices=[0, 1], label="Exercise Induced Angina (exang)"),
        gr.inputs.Number(label="Oldpeak"),
        gr.inputs.Radio(choices=[0, 1, 2], label="Slope of Peak Exercise ST Segment (slope)"),
        gr.inputs.Number(label="Number of Major Vessels (ca)"),
        gr.inputs.Radio(choices=["fixed", "normal", "reversible"], label="Thalassemia (thal)")
    ],
    outputs='text',
    title="Heart Attack Prediction",
    description="Predict the risk of heart attack based on various health indicators."
)

# Launch the interface
iface.launch(debug=True, share=True)


ModuleNotFoundError: No module named 'gradio'